In [36]:
import simpy
env = simpy.Environment()

In [20]:
#definimos una funcion a llamar cuando ocurre un evento
def contador(env):
    while True: #en un ciclo infinto
        print(env.now) #imprime el 'tiempo' actual de la simulacion
        yield env.timeout(3) #espera una unidad de 'tiempo'


env.process(contador(env)) #agregamos un proceso a la simulacion
env.run(until=15) #ejecutamos la simulacion por 5 pasos

0
3
6
9
12


In [21]:
#definimos una funcion a llamar cuando ocurre un evento
def contador(env, nombre, inicio):
    yield env.timeout(inicio) #espera un numero de pasos dado
    while True:
        print(f"{nombre}: {env.now}") #imprime su nombre y el 'tiempo' actual
        yield env.timeout(2) #espera dos pasos de la simulacion

env = simpy.Environment() #inicializamos el env de simulacion
env.process(contador(env, "impares", 1)) #agregamos un proceso a la simulacion
env.process(contador(env, "pares", 2)) #agregamos otro proceso a la simulacion
env.run(until=10) #ejecutamos la simulacion por 6 pasos

impares: 1
pares: 2
impares: 3
pares: 4
impares: 5
pares: 6
impares: 7
pares: 8
impares: 9


In [24]:
#definimos el primer proceso: un contador normal
def primero(env):
    print("inicia primer proceso")
    for i in range(3):
        print(env.now)
        yield env.timeout(1)

#definimos el primer proceso: un contador modificado
def segundo(env, anterior):
    yield anterior #espera a que termine el proceso 'anterior'
    print("inicia segundo proceso")
    while True:
        print(env.now)
        yield env.timeout(1)


primer_proceso = env.process(primero(env))
env.process(segundo(env, primer_proceso))

env.run(until=6) #ejecutamos la simulacion por 6 pasos

inicia primer proceso
0
1
2
inicia segundo proceso
3
4
5


In [26]:
#definimos el proceso principal, que ejecuta procesos secundarios
def principal(env):
    while True:
        yield env.process(secundario("par", env))
        yield env.process(secundario("impar", env))

#definimos el proceso secundario: un contador de un solo paso (con 'nombre')
def secundario(nombre, env):
    print(f"{nombre}: {env.now}")
    yield env.timeout(1)


#agregamos el proceso principal a la simulacion
primer_proceso = env.process(principal(env))

env.run(until=6) #ejecutamos la simulacion por 6 pasos

par: 0
impar: 1
par: 2
impar: 3
par: 4
impar: 5


In [34]:
#definimos el proceso que sera interrumpido
def primero(env):
    try: #las interrupciones se manejan como excepciones (en un bloque try-except)
        while True: #definimos un contador normal
            print(env.now)
            yield env.timeout(1)
    except simpy.Interrupt: #si el proceso es interrumpido, lo reporta y termina
        print("proceso interrumpido")

#definimos el proceso que realizara la interrupcion
def segundo(env, otro):
    yield env.timeout(5)
    otro.interrupt()


primer_proceso = env.process(primero(env))
env.process(segundo(env, primer_proceso))
env.run()

0
1
2
3
4
proceso interrumpido


In [ ]:
#importamos el modulo de SimPy
import simpy

#definimos un proceso de contador que utiliza un recurso en cada iteracion
def contador(env, nombre, recurso):
    while True:
        recurso_utilizado = recurso.request() #solicita el recurso
        yield recurso_utilizado #espera a obtener el recurso
        yield env.timeout(1)
        print(f"{nombre}: {env.now}")
        recurso.release(recurso_utilizado) #libera el recurso para que otro proceso pueda utilizarlo

env = simpy.Environment() #inicializamos el env de simulacion
recurso = simpy.Resource(env, capacity=2) #creamos un recurso, indicando la capacidad

for i in range(4): #creamos un conjunto de procesos y agregamos cada uno a la simulacion
    env.process(contador(env, 'Proceso %d' % i, recurso))

env.run(until=6) #ejecutamos la simulacion por 6 pasos

In [ ]:
#importamos el modulo de SimPy
import simpy

#definimos un proceso de contador que utiliza un recurso en cada iteracion
def contador(env, nombre, recurso):
    while True:
        with recurso.request() as recurso_utilizado: #solicita el recurso
            yield recurso_utilizado #espera a obtener el recurso
            yield env.timeout(1)
            print(f"{nombre}: {env.now}")
            #el recurso es liberado automaticamente al finalizar el bloque with

env = simpy.Environment() #inicializamos el env de simulacion
recurso = simpy.Resource(env, capacity=2) #creamos un recurso, indicando la capacidad


for i in range(4): #creamos un conjunto de procesos y agregamos cada uno a la simulacion
    env.process(contador(env, 'Proceso %d' % i, recurso))

env.run(until=6) #ejecutamos la simulacion por 6 pasos

In [ ]:
#importamos el modulo de SimPy
import simpy

#definimos una clase
class Clase:
    #definicion del constructor de la clase
    def __init__(self, env): #se recibe el env de SimPy como parametro
        self.env = env
        #en el constructor se indica la funcion a ejecutar por defecto
        self.action = env.process(self.contador())
    
    #definimos un contador como funcion de la clase
    def contador(self):
        try: #agregamos un bloque try para poder interrumpir la funcion
            while True:
                print(self.env.now)
                yield self.env.timeout(1)
        except simpy.Interrupt: #si el proceso es interrumpido, lo reporta y termina
            print("proceso interrumpido")


#definimos el proceso que realizara la interrupcion
def interruptor(env, otro):
    yield env.timeout(5)
    otro.action.interrupt() #se debe interrumpir al accion, no el objeto

env = simpy.Environment() #inicializamos el env de simulacion

#creamos un ejemplar de la clase
primer_proceso = Clase(env)
'''Nota: como la instruccion process() esta en el constructor de la clase,
 no hace falta iniciar el proceso, sino que basta con construir el ejemplar '''

#agregamos el proceso interruptor a la simulacion, pasando la referencia al ejemplar
env.process(interruptor(env, primer_proceso))

#ejecutamos la simulacion hasta que termine 
env.run() #(sin limite de tiempo definido por until)

In [ ]:
import simpy

def car(env):
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)

        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)

# Create environment
env = simpy.Environment()

# Start process
env.process(car(env))

# Run simulation for 20 time units
env.run(until=20)
